In [1]:
import talib
import numpy as np
import pandas as pd
import random
from collections import deque

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
# # ----------------------------------------------------------------------------------------------------
# # Get the data
# # ----------------------------------------------------------------------------------------------------
# df1 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2015-2016.csv')
# df1.Date = pd.to_datetime(df1.Date)

# df2 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2016-2017.csv')
# df2.Date = pd.to_datetime(df2.Date)

# df3 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2017-2018.csv')
# df3.Date = pd.to_datetime(df3.Date)

# df4 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2019-2020.csv')
# df4.Date = pd.to_datetime(df4.Date)

# df5 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2020-2021.csv')
# df5.Date = pd.to_datetime(df5.Date)


# dfs1 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2015-2016.csv')
# dfs1.Date = pd.to_datetime(dfs1.Date)

# dfs2 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2016-2017.csv')
# dfs2.Date = pd.to_datetime(dfs2.Date)

# dfs3 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2017-2018.csv')
# dfs3.Date = pd.to_datetime(dfs3.Date)

# dfs4 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2019-2020.csv')
# dfs4.Date = pd.to_datetime(dfs4.Date)

# dfs5 = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
# dfs5.Date = pd.to_datetime(dfs5.Date)

# df = df1.append([df2, df3, df4, df5])
# df.reset_index(drop=True, inplace=True)

# df_small = dfs1.append([dfs2, dfs3, dfs4, dfs5])
# df_small.reset_index(drop=True, inplace=True)

In [4]:
# df

In [5]:
# df_small

In [6]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M15_2020-2021.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_small = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2020-2021.csv')
df_small.Date = pd.to_datetime(df_small.Date)
df_small.reset_index(drop=True, inplace=True)

In [7]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2020-07-30 07:00:00   1.29543   1.29607  1.29540    1.29572   1.29562   
1 2020-07-30 07:05:00   1.29569   1.29591  1.29505    1.29542   1.29588   
2 2020-07-30 07:10:00   1.29541   1.29590  1.29514    1.29559   1.29557   
3 2020-07-30 07:15:00   1.29561   1.29567  1.29510    1.29515   1.29577   
4 2020-07-30 07:20:00   1.29514   1.29643  1.29501    1.29642   1.29529   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.29623  1.29560    1.29589   1.29552   1.29615  1.29550    1.29580  
1   1.29608  1.29522    1.29560   1.29578   1.29600  1.29514    1.29551  
2   1.29606  1.29535    1.29575   1.29549   1.29598  1.29524    1.29567  
3   1.29584  1.29530    1.29533   1.29569   1.29576  1.29520    1.29524  
4   1.29660  1.29518    1.29660   1.29522   1.29652  1.29510    1.29651  
---------------------------------------------------------------------------
                     Date  Bi

In [8]:
print(df_small.head())
print('-' * 75)
print(df_small.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2020-07-30 07:00:00   1.29543   1.29607  1.29540    1.29572   1.29562   
1 2020-07-30 07:05:00   1.29569   1.29591  1.29505    1.29542   1.29588   
2 2020-07-30 07:10:00   1.29541   1.29590  1.29514    1.29559   1.29557   
3 2020-07-30 07:15:00   1.29561   1.29567  1.29510    1.29515   1.29577   
4 2020-07-30 07:20:00   1.29514   1.29643  1.29501    1.29642   1.29529   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.29623  1.29560    1.29589   1.29552   1.29615  1.29550    1.29580  
1   1.29608  1.29522    1.29560   1.29578   1.29600  1.29514    1.29551  
2   1.29606  1.29535    1.29575   1.29549   1.29598  1.29524    1.29567  
3   1.29584  1.29530    1.29533   1.29569   1.29576  1.29520    1.29524  
4   1.29660  1.29518    1.29660   1.29522   1.29652  1.29510    1.29651  
---------------------------------------------------------------------------
                     Date  Bi

In [9]:
def add_fractal(df, i, look_back=2):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan


def add_beep_boop(row):
  macdhist, ema50, mid_low, mid_high, rsi = row[['macdhist', 'ema50', 'Mid_Low', 'Mid_High', 'rsi']]

  if float(macdhist) > 0 and float(mid_low) > float(ema50):
    return 1

  elif float(macdhist) < 0 and float(mid_high) < float(ema50):
    return 2

  else:
    return 0

In [10]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['ema50'] = talib.EMA(df['Mid_Close'], timeperiod=50)
df['adx'] = talib.ADX(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['rsi'] = talib.RSI(df['Mid_Close'], timeperiod=14)
df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
df['beep_boop'] = df.apply(add_beep_boop, axis=1)
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['beep_boop'].value_counts())
print(df['fractal'].value_counts())

0    35299
1    20431
2    18333
Name: beep_boop, dtype: int64
0.0    55662
1.0     9387
2.0     9014
Name: fractal, dtype: int64


In [11]:
value_per_pip = 1.0
max_pips_to_risk = 0.0100
amounts_per_day = [-4, -5, -6]

In [12]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff, max_open_trades=1):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    spread_cutoff /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000
    trades = deque(maxlen=max_open_trades)

    def iterate_thru_small_data(trade):
        def get_end_date(j):
          return df_small.loc[df_small.index[j + 1], 'Date'] if j + 1 < len(df_small) else None

        j = df_small[df_small.Date >= trade['start_date']].index[0]

        nonlocal reward
        nonlocal day_fees
        nonlocal n_losses
        nonlocal n_wins
        nonlocal curr_win_streak
        nonlocal curr_loss_streak
        nonlocal win_streak
        nonlocal loss_streak

        while j < len(df_small):
          curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_date = \
          df_small.loc[df_small.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low',
                                         'Ask_Close', 'Date']]

          if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          # if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
          #     trade['stop_loss'] = curr_bid_high - trade['pips_risked']

          if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          # if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
          #   trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

          if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          j += 1


    i = 2

    while i < len(df):          
        curr_date = df.loc[df.index[i], 'Date']

        # Remove any expired trades -----------------------------------------
        trades_to_remove = []

        for trade in trades:
          if trade['end_date'] is not None and curr_date >= trade['end_date']:
            trades_to_remove.append(trade)

        for trade in trades_to_remove:
          trades.remove(trade)
        # -------------------------------------------------------------------

        ema200 = df.loc[df.index[i - 1], 'ema200']
        adx = df.loc[df.index[i - 1], 'adx']
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        # enough_volatility = (spread / atr) <= atr_percentage
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh - curr_ml)
        beep_boop1 = df.loc[df.index[i - 1], 'beep_boop']
        beep_boop2 = df.loc[df.index[i - 2], 'beep_boop']
        trending = adx >= adx_cutoff

        process_trade = False

        if beep_boop2 == 1 and beep_boop1 == 1 and curr_ml > ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_buys = True

            for trade in trades:
                if trade['trade_type'] != 'buy':
                    all_buys = False
                    break

            if all_buys:
              pullback = None
              u = i - 3

              while u >= 0:
                  if df.loc[df.index[u], 'fractal'] == 1:
                      curr_pullback = float(df.loc[df.index[u], 'Bid_Low'])
                      curr_fractal_distance = curr_ao - curr_pullback

                      if curr_fractal_distance >= fractal_distance:
                          pullback = curr_pullback
                          break

                  u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_ask_open)
                  stop_loss = round(pullback - pullback_cushion, 5)
                  # stop_loss = round(pullback - pullback_cushion, 3)

                  if stop_loss < open_price:
                      curr_pips_to_risk = open_price - stop_loss

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                            'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_buys += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        elif beep_boop2 == 2 and beep_boop1 == 2 and curr_mh < ema200 and trending and enough_volatility and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_sells = True

            for trade in trades:
                if trade['trade_type'] != 'sell':
                    all_sells = False
                    break

            if all_sells:
              pullback = None
              u = i - 3

              while u >= 0:
                  if df.loc[df.index[u], 'fractal'] == 2:
                      curr_pullback = float(df.loc[df.index[u], 'Ask_High'])
                      curr_fractal_distance = curr_pullback - curr_bo

                      if curr_fractal_distance >= fractal_distance:
                          pullback = curr_pullback
                          break

                  u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_bid_open)
                  stop_loss = round(pullback + pullback_cushion, 5)
                  # stop_loss = round(pullback + pullback_cushion, 3)

                  if stop_loss > open_price:
                      curr_pips_to_risk = stop_loss - open_price

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                  'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                  'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_sells += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        if process_trade:
          iterate_thru_small_data(trades[-1])

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak

In [13]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
# risk_reward_ratio_vals = [1.9, 2]
risk_reward_ratio_vals = [2]
pullback_cushion_vals = [5, 10, 15]
# pullback_cushion_vals = [15]
fractal_distances = [5, 10, 15, 20]
# fractal_distances = [15]
bar_lengths = [5, 10]
# bar_lengths = [10]
adx_cutoffs = [20, 25, 30]
# adx_cutoffs = [20]
spread_cutoffs = [2, 3, 4]
# spread_cutoffs = [4]
n_possibilities = len(risk_reward_ratio_vals) * len(pullback_cushion_vals) * len(fractal_distances) * len(bar_lengths) * len(adx_cutoffs) * len(spread_cutoffs)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
  for pullback_cushion in pullback_cushion_vals:
    for fractal_distance in fractal_distances:
      for bar_length in bar_lengths:
        for adx_cutoff in adx_cutoffs:
          for spread_cutoff in spread_cutoffs:
            all_combos.append((risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_pullback_cushion = None
best_last_x_bars_val = None
best_fractal_distance = None
best_bar_length = None
best_adx_cutoff = None
best_spread_cutoff = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff in combos_to_try:
  reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak = run_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff)
  runs_finished += 1

  print(reward)
  print('Num buys: ' + str(n_sells))
  print('Num sells: ' + str(n_buys))
  print('Num trades: ' + str(n_buys + n_sells))
  print('Num wins: ' + str(n_wins))
  print('Num losses: ' + str(n_losses))
  print('Win streak: ' + str(win_streak))
  print('Loss streak: ' + str(loss_streak))
  print('Remaining runs: ' + str(n_runs - runs_finished))

  min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
  if min_item is None or (reward > 0 and reward > min_item['reward']):
    if min_item is not None:
      best_rewards.remove(min_item)
    best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'pullback': pullback_cushion, 'fractal_dist': fractal_distance, 'bar_length': bar_length, 'adx': adx_cutoff, 'spread': spread_cutoff})


  if reward > best_reward:
    best_reward = reward
    best_risk_reward = risk_reward_ratio
    best_pullback_cushion = pullback_cushion
    best_fractal_distance = fractal_distance
    best_bar_length = bar_length
    best_adx_cutoff = adx_cutoff
    best_spread_cutoff = spread_cutoff

  print('Best reward so far: ' + str(best_reward))
  print()

Num runs: 216

-9665.0000000001
Num buys: 139
Num sells: 138
Num trades: 277
Num wins: 80
Num losses: 196
Win streak: 6
Loss streak: 13
Remaining runs: 215
Best reward so far: -9665.0000000001

-6601.499999999809
Num buys: 185
Num sells: 201
Num trades: 386
Num wins: 123
Num losses: 262
Win streak: 4
Loss streak: 11
Remaining runs: 214
Best reward so far: -6601.499999999809

-11445.999999999984
Num buys: 136
Num sells: 156
Num trades: 292
Num wins: 82
Num losses: 209
Win streak: 4
Loss streak: 15
Remaining runs: 213
Best reward so far: -6601.499999999809

-9348.000000000055
Num buys: 135
Num sells: 140
Num trades: 275
Num wins: 81
Num losses: 193
Win streak: 5
Loss streak: 14
Remaining runs: 212
Best reward so far: -6601.499999999809

-7459.9999999999945
Num buys: 122
Num sells: 141
Num trades: 263
Num wins: 79
Num losses: 183
Win streak: 3
Loss streak: 13
Remaining runs: 211
Best reward so far: -6601.499999999809

-8393.999999999829
Num buys: 133
Num sells: 144
Num trades: 277
Num win

KeyboardInterrupt: 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best pullback cushion: ' + str(best_pullback_cushion))
print('Best fractal distance: ' + str(best_fractal_distance))
print('Best bar length: ' + str(best_bar_length))
print('Best spread: ' + str(best_spread_cutoff))
print('Best adx: ' + str(best_adx_cutoff))
print('-----------------------')
print('Top results:')
for entry in best_rewards:
    print(entry)

# 2015-2016:
# {'reward': 8771, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 5}
# {'reward': 7126, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 7170, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 8538, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 5}
# {'reward': 8178, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 8611, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 8158, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 7639, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 7518, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 7370, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 7237, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 8129, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 7668, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 7363, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 7185, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 5}
# {'reward': 7226, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 8329, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 5}
# {'reward': 7496, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 7629, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 5}
# {'reward': 7751, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 2}

# {'reward': 7376, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 7760, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 6834, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 6510, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 5412, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 8885, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 6626, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 7357, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 5481, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 5562, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 5566, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 7028, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 6095, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 5643, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 7088, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 5515, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 6532, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 7481, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 7806, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 6383, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 4}

# 2016-2017:
# {'reward': -10136, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': -1807, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': -2949, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': -1209, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 5}
# {'reward': -3908, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': -4887, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': -6635, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': -2850, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 5}
# {'reward': -1137, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': -5876, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': -8306, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': -1625, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': -6991, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': -6807, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': -9103, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 5}
# {'reward': -6683, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': -1869, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': -2843, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 5, 'adx': 25, 'spread': 5}
# {'reward': -7279, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 5}
# {'reward': -3358, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 5, 'adx': 25, 'spread': 2}

# {'reward': -2929, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': -3842, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': -6707, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': -1960, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': -5228, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': -3368, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 4}
# {'reward': -5301, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': -4141, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': -3613, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': -5481, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': -5206, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': -1130, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': -2935, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': -2166, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': -8298, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': -2046, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': -1669, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': -1775, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': -2849, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 25, 'spread': 4}
# {'reward': 74, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 4}

# 2017-2018:
# {'reward': 3158, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 2705, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 2841, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 5}
# {'reward': 3057, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 2769, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 5}
# {'reward': 3663, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 2867, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 3682, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 2928, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 3276, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 2828, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 4}
# {'reward': 4666, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 2837, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 5}
# {'reward': 3001, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 3286, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 5}
# {'reward': 3467, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 3114, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 2820, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 3218, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 2829, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 2}

# {'reward': 4035, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 5633, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 3540, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 3074, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 3782, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 4290, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 3541, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 3295, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 6881, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 3538, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 3912, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 3767, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 4543, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 3429, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 3319, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 3996, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 3484, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 3385, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 3863, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 5677, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 2}

# 2018-2019:
# {'reward': 7862, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 7617, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 7465, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 7471, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 5}
# {'reward': 7606, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 7372, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 7091, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 7482, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 7312, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 7170, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 5}
# {'reward': 7871, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 7768, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 7435, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 7824, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 7567, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 5}
# {'reward': 7713, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 7702, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 7957, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 7473, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 7037, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 5}

# {'reward': 8475, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 8183, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 8130, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 7736, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 8512, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 9960, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 11075, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 10224, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 10448, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 9670, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 8166, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 10126, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 8166, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 8074, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 8176, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 11038, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 8968, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 9454, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 9289, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 7682, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 4}

# 2019-2020:
# {'reward': 2137, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 35, 'spread': 4}
# {'reward': 1940, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 2068, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 2408, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 2040, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 35, 'spread': 4}
# {'reward': 1879, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 35, 'spread': 4}
# {'reward': 2014, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 5}
# {'reward': 2174, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 35, 'spread': 3}
# {'reward': 1914, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 35, 'spread': 3}
# {'reward': 1978, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 2085, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 1972, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 2354, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 1923, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 5}
# {'reward': 1964, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 2005, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 1995, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 5}
# {'reward': 2314, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 2122, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 2018, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 5}

# {'reward': 7071, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 6441, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 5977, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 7156, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 7744, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 7288, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 6840, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 6389, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 6849, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 5953, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 7242, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 6534, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 6539, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 7109, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 8079, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 6608, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 7253, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 6422, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 7240, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 5827, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 4}


# 2020-2021:
# {'reward': 3265, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 3262, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 2361, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 3047, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 3021, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 2497, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 5}
# {'reward': 3278, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 5}
# {'reward': 2875, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 5}
# {'reward': 3021, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 5}
# {'reward': 2863, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 2350, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 2561, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 3027, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 2690, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 2508, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 4}
# {'reward': 2669, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 5}
# {'reward': 2879, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 2349, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 5}
# {'reward': 2672, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 2650, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 3}

# {'reward': 4214, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 5759, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 5142, 'ratio': 1.9, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 4353, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 5772, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 5468, 'ratio': 1.9, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 5105, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 5113, 'ratio': 1.9, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 5017, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 3480, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 4232, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 5488, 'ratio': 1.9, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 5109, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 3348, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 4}
# {'reward': 4855, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 3626, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 3481, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 4372, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 5004, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 3321, 'ratio': 1.9, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 4}







# 2015-2021:
# {'reward': 9692, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 6157, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 7842, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 6605, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 6066, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 7989, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 7887, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 7219, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 6083, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 7886, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 7712, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 8127, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 8601, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 7982, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 8956, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 9086, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 6807, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 7039, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 10655, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 7367, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 3}



# 2015-2021 (no TL):
# {'reward': 16940, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 11736, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 7118, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 7577, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 8120, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 7582, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 12085, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 7395, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 8113, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 12761, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 8980, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 10375, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 9217, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 6717, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 2}
# {'reward': 7788, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 9970, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 17666, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 9030, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 9407, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 7712, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 10, 'adx': 30, 'spread': 3}

------------ FINAL RESULTS ------------
Best reward: 8885.499999999864
Best risk/reward ratio: 2
Best pullback cushion: 15
Best fractal distance: 15
Best bar length: 10
Best spread: 4
Best adx: 20
-----------------------
Top results:
{'reward': 7376, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 3}
{'reward': 7760, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 4}
{'reward': 6834, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 4}
{'reward': 6510, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
{'reward': 5412, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 3}
{'reward': 8885, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}
{'reward': 6626, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 3}
{'reward': 7357, 'rati

In [ ]:
import json

set1 = { json.dumps({1: 'hi', 2: 'hello'}, sort_keys=True), json.dumps({3: 'foo', 4: 'foo2'}, sort_keys=True) }
set2 = { json.dumps({1: 'hi', 2: 'hello'}, sort_keys=True) }

set1.intersection(set2)

In [17]:
def add_spread(row):
  curr_ao, curr_bo = row[['Ask_Open', 'Bid_Open']]

  return abs(curr_ao - curr_bo)

df_small['spread'] = df_small.apply(add_spread, axis=1)

In [19]:
df_small['spread'].mean()

0.00020055720133577672

In [20]:
df_small['adx'] = talib.ADX(df_small['Mid_High'], df_small['Mid_Low'], df_small['Mid_Close'], timeperiod=14)

In [21]:
df_small['adx'].mean()

24.594419713453895

In [22]:
foo = pd.read_csv(file_path + 'Oanda_Usd_Jpy_M5_2020-2021.csv')
foo.Date = pd.to_datetime(foo.Date)
foo.reset_index(drop=True, inplace=True)
foo['spread'] = foo.apply(add_spread, axis=1)
foo['adx'] = talib.ADX(foo['Mid_High'], foo['Mid_Low'], foo['Mid_Close'], timeperiod=14)

In [23]:
foo['spread'].mean()

0.014943014185830442

In [24]:
foo['adx'].mean()

24.84253478202672